# How was the wheater during session in WUT?

We define exam session time as first two weeks of Feburary and last two weeks of June.

## Instalation of libraries

In [1]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 669.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 494.1 kB/s eta 0:00:00a 0:00:01


## Defining libraries and client config

In [2]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

### POC

In [5]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.22201746504321,
	"longitude": 21.007023575894323,
	"daily": "sunshine_duration",
	"timezone": "Europe/Berlin",
	"start_date": "2024-02-05",
	"end_date": "2024-02-18"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_sunshine_duration = daily.Variables(0).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["sunshine_duration"] = daily_sunshine_duration

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates 52.25°N 21.0°E
Elevation 120.0 m asl
Timezone b'Europe/Berlin' b'CEST'
Timezone difference to GMT+0 7200 s
                        date  sunshine_duration
0  2024-02-04 22:00:00+00:00        2069.141113
1  2024-02-05 22:00:00+00:00           0.000000
2  2024-02-06 22:00:00+00:00           0.000000
3  2024-02-07 22:00:00+00:00        3746.098145
4  2024-02-08 22:00:00+00:00           0.000000
5  2024-02-09 22:00:00+00:00           0.000000
6  2024-02-10 22:00:00+00:00           0.000000
7  2024-02-11 22:00:00+00:00           0.000000
8  2024-02-12 22:00:00+00:00       14933.474609
9  2024-02-13 22:00:00+00:00       29991.185547
10 2024-02-14 22:00:00+00:00        9132.440430
11 2024-02-15 22:00:00+00:00       29597.730469
12 2024-02-16 22:00:00+00:00        4609.540039
13 2024-02-17 22:00:00+00:00       24945.998047
